## Download the data

In [3]:
import geopandas as gpd
import pandas as pd
from tqdm import tqdm

class GetDailyCountyData:
    def __init__(
            self,
            start_date,
            end_date,
            base_url,
            states_shp_url,
            state
    ):
        self.date_range = pd.date_range(start_date, end_date)
        self.base_url = base_url
        state_shp = gpd.read_file(states_shp_url)
        self.state_shp = state_shp[state_shp.NAME == state].iloc[0]
    
    def download(self):
        state_specific_df = pd.DataFrame()
        for d in tqdm(self.date_range):
            csv_url = self.base_url.format(d.year,'%02d' % d.month, '%02d' % d.day)
            df = pd.read_csv(csv_url, engine='pyarrow')
            df_o = gpd.GeoDataFrame(df[['lng_o','lat_o']],  geometry=gpd.points_from_xy(df.lng_o, df.lat_o))
            df_d = gpd.GeoDataFrame(df[['lng_d','lat_d']],  geometry=gpd.points_from_xy(df.lng_d, df.lat_d))
            df = df[(self.state_shp.geometry.contains(df_o.geometry)) & (self.state_shp.geometry.contains(df_d.geometry))]
            state_specific_df = pd.concat([state_specific_df, df], axis=0)
        return state_specific_df

In [15]:
# Get data on the kincaid fire
base_url = "https://github.com/GeoDS/COVID19USFlows-DailyFlows/raw/master/daily_flows/county2county/daily_county2county_{}_{}_{}.csv"
states_shp_url = 'https://github.com/joncutrer/geopandas-tutorial/raw/master/data/usa-states-census-2014.shp'

get_daily_data = GetDailyCountyData('2019-08-02','2019-11-06', base_url, states_shp_url, 'California')
kincaid = get_daily_data.download()
kincaid.to_csv('california_aug_nov_2019.csv')

  4%|▍         | 4/97 [01:04<24:25, 15.76s/it]

datetime.date(2019, 8, 1)